In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
df = pd.read_csv("/content/drive/My Drive/ineuron_DLCVNLP/BiasCorrection_UCI/Bias_correction_ucl.csv")

In [3]:
df.isnull().sum()

station              2
Date                 2
Present_Tmax        70
Present_Tmin        70
LDAPS_RHmin         75
LDAPS_RHmax         75
LDAPS_Tmax_lapse    75
LDAPS_Tmin_lapse    75
LDAPS_WS            75
LDAPS_LH            75
LDAPS_CC1           75
LDAPS_CC2           75
LDAPS_CC3           75
LDAPS_CC4           75
LDAPS_PPT1          75
LDAPS_PPT2          75
LDAPS_PPT3          75
LDAPS_PPT4          75
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [4]:
df.iloc[7743:7752]

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
7743,19.0,2017-08-30,22.6,15.0,25.536045,78.779045,26.596280,17.856040,6.905963,54.030075,0.056634,0.076289,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,37.5776,126.938,75.0924,1.767800,4495.473633,27.1,16.6
7744,20.0,2017-08-30,22.7,15.9,38.216373,94.428116,24.037222,15.772677,6.478060,112.580310,0.038019,0.005380,0.000000e+00,0.000938,0.000000,0.000000,0.000000,0.000000,37.6181,127.004,146.5540,4.729600,4554.965332,26.7,16.3
7745,21.0,2017-08-30,23.1,17.8,24.688997,78.261383,27.812697,18.303014,6.603253,9.614074,0.052902,0.030169,0.000000e+00,0.004380,0.000000,0.000000,0.000000,0.000000,37.5507,127.040,26.2980,0.572100,4456.024414,27.6,17.7
7746,22.0,2017-08-30,22.5,17.4,30.094858,83.690018,26.704905,17.814038,5.768083,82.146707,0.066461,0.024518,5.850000e-07,0.017678,0.000000,0.000000,0.000000,0.000000,37.5102,127.086,21.9668,0.133200,4441.803711,28.0,17.1
7747,23.0,2017-08-30,23.3,17.1,26.741310,78.869858,26.352081,18.775678,6.148918,72.058294,0.030034,0.081035,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,37.5372,126.891,15.5876,0.155400,4443.313965,28.3,18.1
7748,24.0,2017-08-30,23.3,17.7,24.040634,77.294975,27.010193,18.733519,6.542819,47.241457,0.035874,0.074962,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,37.5237,126.909,17.2956,0.222300,4438.373535,28.6,18.8
7749,25.0,2017-08-30,23.2,17.4,22.933014,77.243744,27.939516,18.522965,7.289264,9.090034,0.048954,0.059869,0.000000e+00,0.000796,0.000000,0.000000,0.000000,0.000000,37.5237,126.970,19.5844,0.271300,4451.345215,27.8,17.4
7750,NaN,NaN,20.0,11.3,19.794666,58.936283,17.624954,14.272646,2.882580,-13.603212,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,37.4562,126.826,12.3700,0.098475,4329.520508,17.4,11.3
7751,NaN,NaN,37.6,29.9,98.524734,100.000153,38.542255,29.619342,21.857621,213.414006,0.967277,0.968353,9.837888e-01,0.974710,23.701544,21.621661,15.841235,16.655469,37.6450,127.135,212.3350,5.178230,5992.895996,38.9,29.8


In [5]:
df.iloc[7750, 0] = 1.0
df.iloc[7751, 0] = 2.0

df.iloc[7750, 1] = "2017-08-31"
df.iloc[7751, 1] = "2017-08-31"

In [6]:
df.isnull().sum()

station              0
Date                 0
Present_Tmax        70
Present_Tmin        70
LDAPS_RHmin         75
LDAPS_RHmax         75
LDAPS_Tmax_lapse    75
LDAPS_Tmin_lapse    75
LDAPS_WS            75
LDAPS_LH            75
LDAPS_CC1           75
LDAPS_CC2           75
LDAPS_CC3           75
LDAPS_CC4           75
LDAPS_PPT1          75
LDAPS_PPT2          75
LDAPS_PPT3          75
LDAPS_PPT4          75
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [7]:
train_set = df[~df['Next_Tmax'].isnull()]
test_set = df[df['Next_Tmax'].isnull()]

In [8]:
print(df.shape, train_set.shape, test_set.shape)

(7752, 25) (7725, 25) (27, 25)


In [9]:
test_set.isnull().sum()

station              0
Date                 0
Present_Tmax        10
Present_Tmin        10
LDAPS_RHmin          0
LDAPS_RHmax          0
LDAPS_Tmax_lapse     0
LDAPS_Tmin_lapse     0
LDAPS_WS             0
LDAPS_LH             0
LDAPS_CC1            0
LDAPS_CC2            0
LDAPS_CC3            0
LDAPS_CC4            0
LDAPS_PPT1           0
LDAPS_PPT2           0
LDAPS_PPT3           0
LDAPS_PPT4           0
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [10]:
## Next_Tmax and Next_Tmin are the targets for this dataset

In [11]:
train_set.head()

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,0.203896,0.161697,0.130928,0.0,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,2013-06-30,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,0.251771,0.159444,0.127727,0.0,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,2013-06-30,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,0.257469,0.204091,0.142125,0.0,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,2013-06-30,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,0.226002,0.161157,0.134249,0.0,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,2013-06-30,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,0.249995,0.178892,0.170021,0.0,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5


In [12]:
train_set['station'].nunique()

25

In [13]:
train_set.head(30)

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,0.203896,0.161697,0.130928,0.000000,0.000000,0.000000,0.000000,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,2013-06-30,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,0.251771,0.159444,0.127727,0.000000,0.000000,0.000000,0.000000,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,2013-06-30,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,0.257469,0.204091,0.142125,0.000000,0.000000,0.000000,0.000000,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,2013-06-30,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,0.226002,0.161157,0.134249,0.000000,0.000000,0.000000,0.000000,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,2013-06-30,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,0.249995,0.178892,0.170021,0.000000,0.000000,0.000000,0.000000,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5
5,6.0,2013-06-30,31.9,23.5,52.437126,85.307251,29.219342,23.822613,6.182295,50.231389,0.185279,0.280818,0.232841,0.146363,0.000000,0.000000,0.000000,0.000000,37.5102,127.042,54.6384,0.1457,5873.780762,31.5,24.0
6,7.0,2013-06-30,31.4,24.4,56.287189,81.019760,28.551859,24.238467,5.587135,125.110007,0.389600,0.333570,0.270419,0.145705,0.000000,0.000000,0.000000,0.000000,37.5776,126.838,12.3700,0.0985,5849.233398,30.9,23.4
7,8.0,2013-06-30,32.1,23.6,52.326218,78.004539,28.851982,23.819054,6.104417,42.011547,0.357856,0.344928,0.272387,0.143841,0.000000,0.000000,0.000000,0.000000,37.4697,126.910,52.5180,1.5629,5863.992188,31.1,22.9
8,9.0,2013-06-30,31.4,22.0,55.338791,80.784607,28.426975,23.332373,6.017135,85.110971,0.408766,0.344800,0.264980,0.148911,0.000000,0.000000,0.000000,0.000000,37.4967,126.826,50.9312,0.4125,5876.901367,31.3,21.6
9,10.0,2013-06-30,31.6,20.5,56.651203,86.849632,27.576705,22.527018,6.518841,63.006075,0.348292,0.306255,0.245254,0.135443,0.000000,0.000000,0.000000,0.000000,37.4562,126.955,208.5070,5.1782,5893.608398,30.5,21.0


In [14]:
# By scanning we will fill the next day Tmax and next day Tmin to the present day value for each station
# or we can also forward fill the data since it is a temporal data

In [15]:
def combine(train,test):
  train['is_train'] = 1
  test['is_train'] = 0
  combined = pd.concat([train,test],axis=0)
  return combined

combined = combine(train_set,test_set)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
combined.shape

(7752, 26)

In [17]:
combined['Date'] = pd.to_datetime(combined['Date'])

In [18]:
combined.sort_values(by=["station",'Date'],inplace=True)

In [19]:
non_targets = combined.drop(['Next_Tmax',"Next_Tmin"],axis=1)

In [20]:
non_targets.fillna(method="ffill",inplace=True)

In [21]:
combined = pd.concat([non_targets, combined[['Next_Tmax','Next_Tmin']]],axis=1)

In [22]:
combined.head()

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,is_train,Next_Tmax,Next_Tmin
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,0.203896,0.161697,0.130928,0.000000,0.000000,0.000000,0.000000,37.6046,126.991,212.335,2.785,5992.895996,1,29.1,21.2
25,1.0,2013-07-01,28.3,21.2,72.798576,97.642792,25.276716,21.142562,15.608045,64.914946,0.615612,0.843199,0.810455,0.622040,0.804222,9.933111,0.237004,0.848082,37.6046,126.991,212.335,2.785,5987.718750,1,24.8,18.7
50,1.0,2013-07-02,24.4,20.6,55.647278,98.370041,27.785497,19.561770,11.168633,74.625920,0.312105,0.273683,0.030997,0.010617,0.000503,0.000000,0.000000,0.000000,37.6046,126.991,212.335,2.785,5981.979492,1,28.1,17.8
75,1.0,2013-07-03,27.9,17.9,76.017967,95.990532,28.125651,21.982579,10.546499,39.809905,0.187604,0.390579,0.438748,0.789554,0.000000,0.000000,0.000000,0.051695,37.6046,126.991,212.335,2.785,5975.676270,1,25.2,20.8
100,1.0,2013-07-04,24.9,21.5,55.888565,97.402481,30.458252,22.077078,7.393145,63.599636,0.814980,0.430494,0.160574,0.001116,0.096997,0.021871,0.000000,0.000000,37.6046,126.991,212.335,2.785,5968.809082,1,28.0,19.7


In [23]:
combined.isnull().sum()

station              0
Date                 0
Present_Tmax         0
Present_Tmin         0
LDAPS_RHmin          0
LDAPS_RHmax          0
LDAPS_Tmax_lapse     0
LDAPS_Tmin_lapse     0
LDAPS_WS             0
LDAPS_LH             0
LDAPS_CC1            0
LDAPS_CC2            0
LDAPS_CC3            0
LDAPS_CC4            0
LDAPS_PPT1           0
LDAPS_PPT2           0
LDAPS_PPT3           0
LDAPS_PPT4           0
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
is_train             0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [24]:
import re
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [25]:
add_datepart(combined, "Date")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


In [26]:
combined.head()

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,is_train,Next_Tmax,Next_Tmin,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,0.203896,0.161697,0.130928,0.000000,0.000000,0.000000,0.000000,37.6046,126.991,212.335,2.785,5992.895996,1,29.1,21.2,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400
25,1.0,28.3,21.2,72.798576,97.642792,25.276716,21.142562,15.608045,64.914946,0.615612,0.843199,0.810455,0.622040,0.804222,9.933111,0.237004,0.848082,37.6046,126.991,212.335,2.785,5987.718750,1,24.8,18.7,2013,7,27,1,0,182,False,True,False,True,False,False,1372636800
50,1.0,24.4,20.6,55.647278,98.370041,27.785497,19.561770,11.168633,74.625920,0.312105,0.273683,0.030997,0.010617,0.000503,0.000000,0.000000,0.000000,37.6046,126.991,212.335,2.785,5981.979492,1,28.1,17.8,2013,7,27,2,1,183,False,False,False,False,False,False,1372723200
75,1.0,27.9,17.9,76.017967,95.990532,28.125651,21.982579,10.546499,39.809905,0.187604,0.390579,0.438748,0.789554,0.000000,0.000000,0.000000,0.051695,37.6046,126.991,212.335,2.785,5975.676270,1,25.2,20.8,2013,7,27,3,2,184,False,False,False,False,False,False,1372809600
100,1.0,24.9,21.5,55.888565,97.402481,30.458252,22.077078,7.393145,63.599636,0.814980,0.430494,0.160574,0.001116,0.096997,0.021871,0.000000,0.000000,37.6046,126.991,212.335,2.785,5968.809082,1,28.0,19.7,2013,7,27,4,3,185,False,False,False,False,False,False,1372896000


In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
sc = StandardScaler()

In [29]:
combined.columns

Index(['station', 'Present_Tmax', 'Present_Tmin', 'LDAPS_RHmin', 'LDAPS_RHmax',
       'LDAPS_Tmax_lapse', 'LDAPS_Tmin_lapse', 'LDAPS_WS', 'LDAPS_LH',
       'LDAPS_CC1', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4', 'LDAPS_PPT1',
       'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4', 'lat', 'lon', 'DEM', 'Slope',
       'Solar radiation', 'is_train', 'Next_Tmax', 'Next_Tmin', 'Year',
       'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'Elapsed'],
      dtype='object')

In [30]:
columns_to_scale = ['Present_Tmax', 'Present_Tmin', 'LDAPS_RHmin', 'LDAPS_RHmax',
       'LDAPS_Tmax_lapse', 'LDAPS_Tmin_lapse', 'LDAPS_WS', 'LDAPS_LH',
       'LDAPS_CC1', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4', 'LDAPS_PPT1',
       'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4', 'lat', 'lon', 'DEM', 'Slope',
       'Solar radiation']

scaled_array = sc.fit_transform(combined[columns_to_scale].values)


In [31]:
scaled_df = pd.DataFrame(scaled_array,columns=columns_to_scale)
combined = pd.concat([combined.drop(columns_to_scale,axis=1),scaled_df],axis=1)

In [32]:
train_set = combined[combined['is_train'] == 1]
test_set = combined[combined['is_train'] == 0]

In [33]:
train_set.head()

,station,is_train,Next_Tmax,Next_Tmin,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation
0,1.0,1,29.1,21.2,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400,-0.357343,-0.755260,0.101623,0.380569,-0.530080,-0.225469,-0.126533,0.202923,-0.523341,-0.592445,-0.623047,-0.658879,-0.310862,-0.274785,-0.239870,-0.222227,1.189286,-0.005,2.772243,1.115004,1.517935
1,2.0,1,30.5,22.5,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400,-0.491820,-0.838108,1.095604,1.289481,-1.480497,-1.019492,3.908619,0.068416,0.930709,1.896330,1.973543,1.273195,0.088288,5.380653,-0.035729,0.484168,1.189286,-0.005,2.772243,1.115004,1.505870
2,3.0,1,31.1,23.9,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400,-1.802971,-1.086653,-0.076657,1.390763,-0.628134,-1.692740,1.870460,0.356324,-0.225578,-0.320766,-1.146160,-1.132195,-0.310612,-0.274785,-0.239870,-0.222227,1.189286,-0.005,2.772243,1.115004,1.492496
3,4.0,1,31.7,24.3,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400,-0.626297,-2.205107,1.315644,1.059377,-0.512565,-0.661735,1.584835,-0.675890,-0.699900,0.134303,0.485824,1.932212,-0.310862,-0.274785,-0.239870,-0.179169,1.189286,-0.005,2.772243,1.115004,1.477808
4,5.0,1,31.2,22.5,2013,6,26,30,6,181,True,False,True,False,False,False,1372550400,-1.634875,-0.713836,-0.060166,1.256014,0.279940,-0.621489,0.137112,0.029420,1.690254,0.289692,-0.627543,-1.169571,-0.262721,-0.262332,-0.239870,-0.222227,1.189286,-0.005,2.772243,1.115004,1.461805


# Model

In [34]:
from tensorflow.keras import Sequential

In [36]:
from tensorflow.keras.layers import Dense

In [57]:
model = Sequential()
model.add(Dense(13, input_dim=34, kernel_initializer='normal', activation='relu'))
model.add(Dense(13, kernel_initializer='normal', activation='relu'))
model.add(Dense(13, kernel_initializer='normal', activation='relu'))
model.add(Dense(13, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

In [58]:
X = train_set.drop(['station','is_train','Next_Tmax','Next_Tmin'],axis=1)
y = train_set[['Next_Tmax','Next_Tmin']]

In [59]:
mycols = ['Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start','Is_year_end', 'Is_year_start']
for col in mycols:
  X[col] = X[col].replace(True,1)
  X[col] = X[col].replace(False,0)

In [61]:
model.fit(X.values,y.values, epochs=100,batch_size=200)

Epoch 1/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4684
Epoch 2/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4647
Epoch 3/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4640
Epoch 4/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4632
Epoch 5/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4644
Epoch 6/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4661
Epoch 7/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4649
Epoch 8/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4651
Epoch 9/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4660
Epoch 10/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4666
Epoch 11/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4654
Epoch 12/100
39/39 [==============================] - 0s 1ms/step - loss: 8.4681
Epoch 13/100
39/39 [=================

In [62]:
X_test = test_set.drop(['station','is_train','Next_Tmax','Next_Tmin'],axis=1)
y_test = test_set[['Next_Tmax','Next_Tmin']]

In [64]:
mycols = ['Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start','Is_year_end', 'Is_year_start']
for col in mycols:
  X_test[col] = X_test[col].replace(True,1)
  X_test[col] = X_test[col].replace(False,0)

In [66]:
preds = model.predict(X_test.values)

In [67]:
test_set[['Next_Tmax','Next_Tmin']] = preds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [68]:
test_set

,station,is_train,Next_Tmax,Next_Tmin,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation
831,7.0,0,28.903372,21.804266,2013,8,31,2,4,214,False,False,False,False,False,False,1375401600,0.247803,1.605920,0.954621,0.740021,-0.247315,0.817940,-0.149301,-0.233717,1.844065,1.849231,1.832707,1.044394,-0.200080,3.092861,0.519896,-0.182933,0.653021,0.838510,-0.526218,-0.723133,0.538797
856,7.0,0,28.905186,21.805630,2013,8,31,3,5,215,False,False,False,False,False,False,1375488000,1.458096,1.067405,-1.296236,-0.906486,1.975597,0.984399,-0.974132,0.161351,-1.173855,-0.877276,-0.563865,0.542700,-0.310862,-0.274785,-0.211509,9.418070,0.653021,0.838510,-0.526218,-0.723133,-1.022698
1073,24.0,0,28.919720,21.816578,2013,8,32,11,6,223,False,False,False,False,False,False,1376179200,-0.592678,-1.832289,0.602416,-0.647014,-1.353126,-1.447480,-0.819703,0.088896,-0.731966,-0.195630,1.142220,1.803113,-0.310862,-0.274785,-0.239870,-0.222227,1.991696,0.385280,-0.297588,0.932424,0.801879
2465,16.0,0,29.571550,22.308084,2014,8,32,5,1,217,False,False,False,False,False,False,1407196800,-1.500398,-0.133897,1.304805,0.504604,-2.016127,-0.986692,1.415780,-0.263262,1.401725,1.737110,2.142313,2.276347,-0.310862,3.055732,8.879560,3.547475,-1.490051,-1.024766,-0.172266,0.223191,-0.746962
3696,22.0,0,30.210649,22.789997,2015,7,30,23,3,204,False,False,False,False,False,False,1437609600,1.760670,1.398799,-0.769391,-1.230168,1.760869,2.122667,-0.249341,-0.952671,0.202649,0.606932,-0.571203,-0.602351,-0.310862,-0.274785,-0.239870,0.041962,0.118743,-0.042770,1.294304,-0.484508,0.021186
3714,15.0,0,30.212471,22.791355,2015,7,30,24,4,205,False,False,False,False,False,False,1437696000,-0.525440,-0.051048,1.414672,0.559470,-0.434359,0.189287,2.661440,-0.209706,1.448588,1.059851,1.857264,-0.640592,1.549943,-0.274785,1.474892,-0.222227,0.118743,-0.042770,1.294304,-0.484508,-1.323759
3739,15.0,0,30.214289,22.792711,2015,7,30,25,5,206,False,False,False,False,False,False,1437782400,-1.063348,0.363194,0.998811,0.457385,-0.824072,0.520776,-0.857059,-1.105588,0.990544,1.141022,1.089804,0.589931,-0.166233,-0.233647,-0.239870,-0.222227,0.653021,1.153252,-0.037504,1.043125,0.803594
3886,12.0,0,30.225197,22.800926,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,0.516757,0.280345,-0.837489,0.051191,0.802281,0.224917,-0.585776,-0.962071,-0.095684,0.729159,-0.686735,-0.709938,-0.310862,-0.273925,-0.239870,-0.222227,0.653021,1.153252,-0.037504,1.043125,-0.398569
4205,6.0,0,30.248810,22.818737,2015,8,33,13,3,225,False,False,False,False,False,False,1439424000,0.953808,0.653163,-0.373175,-1.428927,0.078057,0.616768,-0.821106,-1.407244,-0.582307,-0.355930,1.298502,1.679820,-0.310862,-0.274785,-0.239870,-0.222227,-0.953787,-0.810742,-0.569309,-0.466338,-1.049366
4230,6.0,0,30.250587,22.820091,2015,8,33,14,4,226,False,False,False,False,False,False,1439510400,0.247803,-0.630987,-0.355731,-0.481652,-0.148710,0.308462,-0.176620,-1.307907,0.681057,-0.051544,0.744134,0.782151,-0.296047,-0.274785,-0.239870,-0.222227,-0.953787,-0.810742,-0.569309,-0.466338,0.932661
